# Retrieval  Augmented Generation (RAG)

### What problem it solves ?
* Limited knowledge in LLM.
* Context  window problem (fixed number of tokens sent to the LLM)
* Hallucinations in LLM .

While  evaluating RAG , we  need to separately evaluate Retriever component and Generator Component .
We need Two things :
* Evaluation metric
* Evaluation Dataset


So ,  I will be using RAGAs  (Retrieval-Augmented Generation Assessment)  framework for evaluation .



###### Retrieval Component
* Context precision - It measures the signal-to-noise ratio of the retrieved context. This metric is computed using the question and the contexts
* Context Recall - This metric states that if all the relevant information required to answer the question was retrieved. This metric is computed based on the ground_truth .

###### Generator Component
* Faithfulness - It measures the factual accuracy of the generated answer.The number of correct statements from the given contexts is divided by the total number of statements in the generated answer. This metric uses the question, contextsand the answer.
* Answer relevancy - It measures how relevant the generated answer is to the question. This metric is computed using the question and the answer. For example, the answer “France is in western Europe.” to the question “Where is France and what is it’s capital?” would achieve a low answer relevancy because it only answers half of the question.


All metrics are scaled to the range [0, 1], with higher values indicating a better performance.


In [23]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data2").load_data()
vector_index = VectorStoreIndex.from_documents(documents)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [5]:
questions = [
    "What is Artificial Intelligence (AI)?",
    "What are the three types of AI based on capabilities?",
    "What are the key challenges of AI?"
]

ground_truths = [
    "AI is the branch of computer science that enables systems to perform tasks requiring human intelligence, such as problem-solving, reasoning, and learning.",
    "The three types are Narrow AI (designed for specific tasks), General AI (human-like intelligence), and Superintelligent AI (surpassing human intelligence).",
    "Key challenges include bias in data, privacy concerns, job displacement, maintaining human control, and high energy consumption."
]

In [11]:

from datasets import Dataset

answers = []
contexts = []
for q in questions:
    response = query_engine.query(q)
    answers.append(response.response)
    contexts.append([sn.get_content() for sn in response.source_nodes])
data = {
    "question": questions,
    "contexts": contexts,
    "answer": answers,
    "ground_truth": ground_truths,
}
dataset = Dataset.from_dict(data)


In [12]:
dataset

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 3
})

In [21]:
from ragas import evaluate
from ragas.metrics import (
 faithfulness,
 answer_relevancy,
 context_recall,
 context_precision,
)
result = evaluate(
 dataset = dataset, 
 metrics=[
 context_precision,
 context_recall,
 faithfulness,
 answer_relevancy,
 ],
)
df = result.to_pandas()

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [22]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What is Artificial Intelligence (AI)?,[Artificial Intelligence (AI) is a branch of c...,Artificial Intelligence (AI) is a branch of co...,AI is the branch of computer science that enab...,1.0,1.0,1.0,0.934482
1,What are the three types of AI based on capabi...,[Artificial Intelligence (AI) is a branch of c...,The three types of AI based on capabilities ar...,The three types are Narrow AI (designed for sp...,1.0,1.0,1.0,1.000000
2,What are the key challenges of AI?,[Artificial Intelligence (AI) is a branch of c...,The key challenges of AI include bias and fair...,"Key challenges include bias in data, privacy c...",1.0,1.0,1.0,1.000000
